In [ ]:
import sys
sys.path.append("..")

from typing import List, Dict, Callable, Optional
from datetime import datetime, timedelta
from IPython.display import display

import numpy as np
import pandas as pd

from sesto.constants import CURRENCIES, MT5Timeframe
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.backtester import Backtester, Trade

In [ ]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

In [ ]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [ ]:
TIMEFRAMES = [MT5Timeframe.M5, MT5Timeframe.M15, MT5Timeframe.H1, MT5Timeframe.H4]
DATA_FROM_DATE = datetime(2024, 4, 5)
DATA_TO_DATE = datetime(2024, 9, 20)

MA_PERIODS = [7, 14, 21]

CSM_PERIOD = 21

CSM_TOP_THRESHOLD = 65
CSM_LOW_THRESHOLD = 35
CSM_TOP_REVERSAL_THRESHOLD = 85
CSM_LOW_REVERSAL_THRESHOLD = 15

HISTOGRAM_THRESHOLD = 0.4


In [ ]:
BACKTEST_FROM_DATE = datetime(2024, 1, 1)
BACKTEST_TO_DATE = datetime(2024, 6, 1)

INITIAL_CAPITAL = 10000

CAPITAL_PER_TRADE = 200  # $200 per trade
TRANSACTION_COST = 0.0001
SLIPPAGE = 0.0001
TRADE_SL_MULTIPLIER = 0.0025
RR_MULTIPLIER = 2
LEVERAGE = 500

In [ ]:
mtd.fetch_data_all_timeframes(PAIRS, TIMEFRAMES, DATA_FROM_DATE, DATA_TO_DATE)

In [ ]:
# Remove symbols with no data for any timeframe
for timeframe in TIMEFRAMES:
    empty_symbols = [symbol for symbol, df in mtd.data[timeframe].items() if df is None or df.empty]
    for symbol in empty_symbols:
        del mtd.data[timeframe][symbol]
        if symbol in PAIRS:
            PAIRS.remove(symbol)

print(f"Remaining pairs after removing empty data: {len(PAIRS)}")

In [ ]:
import plotly.graph_objects as go

# Calculate expected number of rows for each timeframe
expected_rows = {}
for timeframe in TIMEFRAMES:
    if timeframe == MT5Timeframe.M5:
        minutes_per_candle = 5
    elif timeframe == MT5Timeframe.M15:
        minutes_per_candle = 15
    elif timeframe == MT5Timeframe.H1:
        minutes_per_candle = 60
    elif timeframe == MT5Timeframe.H4:
        minutes_per_candle = 240
    
    total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
    expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

# Create a DataFrame to store the normalized row counts
heatmap_data = []

for timeframe in TIMEFRAMES:
    for symbol in PAIRS:
        if symbol in mtd.data[timeframe]:
            actual_rows = len(mtd.data[timeframe][symbol])
            normalized_value = actual_rows / expected_rows[timeframe]
            heatmap_data.append({
                'Timeframe': timeframe.name,
                'Symbol': symbol,
                'Normalized Row Count': normalized_value
            })

heatmap_df = pd.DataFrame(heatmap_data)

# Aggregate data to handle potential duplicates
heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Row Count'].mean().reset_index()

# Pivot the DataFrame for the heatmap
heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Row Count')

# Create the heatmap using Plotly with a dark theme
fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    colorscale='Twilight',
    colorbar=dict(
        title='Normalized Row Count',
        titleside='right',
        thickness=18,
        tickfont=dict(color='white'),
        titlefont=dict(color='white')
    ),
    text=heatmap_pivot.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 12, "color": "black"},
))

# Update layout with dark theme and improved spacing
fig.update_layout(
    title={
        'text': 'Normalized Row Count Heatmap: Actual vs Expected',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    yaxis_title='Symbol',
    height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
    width=900,
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)',
    font=dict(color='white', family="Inter"),
    title_font=dict(color='white', size=16),
    xaxis=dict(
        tickfont=dict(color='white', size=12),
        title_font=dict(color='white', size=16),
        side='top',
        tickangle=-45
    ),
    yaxis=dict(
        tickfont=dict(color='white', size=12),
        title_font=dict(color='white', size=16),
        automargin=True,
    ),
    margin=dict(l=125, r=50, t=100, b=50)
)

# Show the plot
fig.show()

In [ ]:
from sesto.indicators import SMA, EMA, RSI, ROC, MACD, BB, ATR

for timeframe, pairs_data in mtd.data.items():
    for pair, df in pairs_data.items():
        if df is not None and not df.empty:
            for period in MA_PERIODS:
                SMA(df, period)
                EMA(df, period)
                ROC(df, period)
                RSI(df, period)
                BB(df, period, 2)
                ATR(df, period)
    
            df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2

display(mtd.data[MT5Timeframe.H1]['EURUSD'].head())



In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Calculate expected number of rows for each timeframe
expected_rows = {}
for timeframe in TIMEFRAMES:
    if timeframe == MT5Timeframe.M5:
        minutes_per_candle = 5
    elif timeframe == MT5Timeframe.M15:
        minutes_per_candle = 15
    elif timeframe == MT5Timeframe.H1:
        minutes_per_candle = 60
    elif timeframe == MT5Timeframe.H4:
        minutes_per_candle = 240
    
    total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
    expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

# Create a DataFrame to store the normalized row counts
heatmap_data = []

for timeframe in TIMEFRAMES:
    for symbol in PAIRS:
        if symbol in mtd.data[timeframe]:
            df = mtd.data[timeframe][symbol]
            if df is not None and not df.empty:
                # Check for the presence of indicators
                indicator_columns = [col for col in df.columns if col.startswith(('sma-', 'ema-', 'rsi-', 'roc-', 'macd', 'bb'))]
                if indicator_columns:
                    actual_rows = df[indicator_columns].dropna().shape[0]
                    normalized_value = actual_rows / expected_rows[timeframe]
                    heatmap_data.append({
                        'Timeframe': timeframe.name,
                        'Symbol': symbol,
                        'Normalized Indicator Count': normalized_value
                    })

heatmap_df = pd.DataFrame(heatmap_data)

# Aggregate data to handle potential duplicates
heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Indicator Count'].mean().reset_index()

# Pivot the DataFrame for the heatmap
heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Indicator Count')

# Create the heatmap using Plotly with a dark theme
fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    colorscale='Twilight',
    colorbar=dict(
        title='Normalized Indicator Count',
        titleside='right',
        thickness=18,
        tickfont=dict(color='white'),
        titlefont=dict(color='white')
    ),
    text=heatmap_pivot.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 12, "color": "black"},
))

# Update layout with dark theme and improved spacing
fig.update_layout(
    title={
        'text': 'Normalized Indicator Count Heatmap: Actual vs Expected',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    yaxis_title='Symbol',
    height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
    width=900,
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)',
    font=dict(color='white', family="Inter"),
    title_font=dict(color='white', size=16),
    xaxis=dict(
        tickfont=dict(color='white', size=12),
        title_font=dict(color='white', size=16),
        side='top',
        tickangle=-45
    ),
    yaxis=dict(
        tickfont=dict(color='white', size=12),
        title_font=dict(color='white', size=16),
        automargin=True,
    ),
    margin=dict(l=125, r=50, t=100, b=50)
)

# Show the plot
fig.show()

In [ ]:
for timeframe in TIMEFRAMES:
    for currency in CURRENCIES:
        # Initialize DataFrames to hold PSM values for the current currency as base and quote
        base_psm_df = pd.DataFrame()
        quote_psm_df = pd.DataFrame()
        
        # Iterate over all pair data for the current timeframe
        for symbol, df in mtd.data[timeframe].items():
            if df is None or df.empty:
                continue
            
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency:
                # Use PSM as is for base currency
                base_psm_df[symbol] = df['psm']
                base_psm_df['time'] = df['time']
            elif currency == quote_currency:
                # Invert PSM for quote currency
                quote_psm_df[symbol] = -df['psm']
                quote_psm_df['time'] = df['time']
        
        # Calculate base CSM
        if not base_psm_df.empty:
            base_csm_df = base_psm_df.groupby('time').mean()
            base_csm_df['csm'] = base_csm_df.mean(axis=1)
            base_csm_df['csm'] = base_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = base_csm_df['csm'].min()
            max_csm = base_csm_df['csm'].max()
            base_csm_df['csm'] = 100 * (base_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for base CSM
            MACD(base_csm_df, 'csm', 12, 26, 9)
        
        # Calculate quote CSM
        if not quote_psm_df.empty:
            quote_csm_df = quote_psm_df.groupby('time').mean()
            quote_csm_df['csm'] = quote_csm_df.mean(axis=1)
            quote_csm_df['csm'] = quote_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = quote_csm_df['csm'].min()
            max_csm = quote_csm_df['csm'].max()
            quote_csm_df['csm'] = 100 * (quote_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for quote CSM
            MACD(quote_csm_df, 'csm', 12, 26, 9)
        
        # Add CSM and MACD values to all relevant symbols in mtd.data
        for symbol in mtd.data[timeframe]:
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency and not base_csm_df.empty:
                mtd.data[timeframe][symbol]['base_csm'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['csm'])
                mtd.data[timeframe][symbol]['base_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd'])
                mtd.data[timeframe][symbol]['base_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['base_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-histogram'])
            elif currency == quote_currency and not quote_csm_df.empty:
                mtd.data[timeframe][symbol]['quote_csm'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['csm'])
                mtd.data[timeframe][symbol]['quote_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd'])
                mtd.data[timeframe][symbol]['quote_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['quote_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-histogram'])

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

# Choose a random timeframe and symbol
random_timeframe = random.choice(TIMEFRAMES)
random_symbol = random.choice(PAIRS)

# Get the data for the chosen timeframe and symbol
df = mtd.data[random_timeframe][random_symbol]

# Create subplots
fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Close Price + SMA', 'CSM Values', 'Base CSM MACD', 'Quote CSM MACD', 'RSI', 'ROC'))

# 1. Close Price + SMA
fig.add_trace(go.Scatter(x=df['time'], y=df['close'], name='Close Price'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['time'], y=df['sma-14'], name='SMA-14'), row=1, col=1)

# 2. CSM Values
fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm'], name='Base CSM'), row=2, col=1)
fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm'], name='Quote CSM'), row=2, col=1)

# 3. Base CSM MACD
fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd'], name='Base MACD'), row=3, col=1)
fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd_signal'], name='Base Signal'), row=3, col=1)
fig.add_bar(x=df['time'], y=df['base_csm_macd_histogram'], name='Base Histogram', row=3, col=1)

# 4. Quote CSM MACD
fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd'], name='Quote MACD'), row=4, col=1)
fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd_signal'], name='Quote Signal'), row=4, col=1)
fig.add_bar(x=df['time'], y=df['quote_csm_macd_histogram'], name='Quote Histogram', row=4, col=1)

# 5. RSI
fig.add_trace(go.Scatter(x=df['time'], y=df['rsi-14'], name='RSI-14'), row=5, col=1)

# 6. ROC
fig.add_trace(go.Scatter(x=df['time'], y=df['roc-14'], name='ROC-14'), row=6, col=1)

# Update layout
fig.update_layout(
    height=1500, 
    width=1600, 
    title_text=f"Chart for {random_symbol} ({random_timeframe.name})", 
    template='plotly_dark'
)

# Update x-axis to include range slider
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
    row=6, col=1  # Apply to the bottom subplot
)

# Hide rangeslider for all subplots except the bottom one
for i in range(1, 6):
    fig.update_xaxes(rangeslider_visible=False, row=i, col=1)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="CSM", row=2, col=1)
fig.update_yaxes(title_text="Base MACD", row=3, col=1)
fig.update_yaxes(title_text="Quote MACD", row=4, col=1)
fig.update_yaxes(title_text="RSI", row=5, col=1)
fig.update_yaxes(title_text="ROC", row=6, col=1)

# Show the plot
# fig.show()

In [ ]:
class MyStrategy(Backtester):
    def entry_condition(
        self, 
        symbol: str, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe,
    ) -> Optional[Dict]:
        # Check if we already have an open trade for this symbol
        if any(trade.symbol == symbol for trade in open_trades):
            return None

        return None

    def exit_condition(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ) -> bool:
        pass

    def trailing_stop(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ):
        pass

In [ ]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    main_timeframe=MT5Timeframe.H1,  # Specify the main timeframe here
    transaction_cost=TRANSACTION_COST, 
    slippage=SLIPPAGE,
    leverage=LEVERAGE
)

backtest.run()

In [ ]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log

In [ ]:
performance_report = backtest.generate_report()
performance_report